# 데이터 정리
- data_1901_2004_classified.ftr -> 데이터 정리 -> data_1901_2004_classified_2.ftr

# 데이터 불러오기
- 새 업종분류가 적용된 데이터

In [1]:
from google.colab import drive
drive.mount('/content/drive') #force_remount=True
import pandas as pd
import numpy as np

data_dir = '/content/drive/My Drive//영빈/data/'
df = pd.read_feather(data_dir + 'data_1901_2004_classified.ftr') # 90971056 rows
                     
df

Mounted at /content/drive


,기준년월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,이용금액,이용건수,업종
0,201901,강원도,강릉시,여성,40대,B2,1070000.0,4,교육서비스
1,201901,강원도,강릉시,남성,20대,B2,48000.0,4,개인미용
2,201901,강원도,강릉시,남성,50대,B4,735000.0,4,주거
3,201901,강원도,강릉시,남성,30대,B4,1041289.0,16,개인운송
4,201901,강원도,강릉시,여성,40대,B4,426766.0,13,개인운송
...,...,...,...,...,...,...,...,...,...
90971051,202004,충청북도,충주시,여성,50대,B5,303000.0,17,음식점
90971052,202004,충청북도,충주시,여성,60대 이상,B1,100000.0,5,음식점
90971053,202004,충청북도,충주시,여성,60대 이상,B2,89000.0,7,음식점
90971054,202004,충청북도,충주시,여성,60대 이상,B3,636000.0,29,음식점


# 결측치 삭제


In [2]:
temp1 = df.shape

# row 삭제: 연령대/가구/소득 x값 
df.drop(df[df.연령대별 == 'x'].index, inplace=True)
df.drop(df[df.연평균소득추정 == 'x'].index, inplace=True)

# 결측치 제거
df.dropna(axis=0, inplace=True)

temp2 = df.shape
temp3 = temp1[0]-temp2[0]  

record1_1 = '데이터 shape 변화: {0} -> {1}'.format(temp1, temp2)
record1_2 = '삭제된 row 수: {0}개, 전체의 {1}%'.format(temp3, round(temp3/temp1[0]*100),1)

print('결측치 제거')
print(record1_1)
print(record1_2)

결측치 제거
데이터 shape 변화: (90971056, 9) -> (88656835, 9)
삭제된 row 수: 2314221개, 전체의 3%


# 년,월 추가
- 기준년월 사용해 년,월 col 생성 후 메모리 최적화

In [3]:
df['날짜'] = df['기준년월']/100

df.drop(columns=['기준년월'], inplace=True)

import numpy as np
date = np.array(df['날짜'])
month, year = np.modf(date)

df.drop(columns=['날짜'], inplace=True)

df['년'] = year
df['월'] = month*100

df['년'] = df['년'].astype(int)
df['월'] = df['월'].astype(int) + 1

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in ['년','월']:
        col_type = df[col].dtype
        if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)                    
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                    elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                        df[col] = df[col].astype(np.uint64)
                elif str(col_type)[:5] == 'float':
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

        elif col_type == object:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

df = reduce_mem_usage(df)

df

Memory usage of dataframe is 3635.65 MB
Memory usage after optimization is: 2536.51 MB
Decreased by 30.2%


,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,이용금액,이용건수,업종,년,월
0,강원도,강릉시,여성,40대,B2,1070000.0,4,교육서비스,2019,1
1,강원도,강릉시,남성,20대,B2,48000.0,4,개인미용,2019,1
2,강원도,강릉시,남성,50대,B4,735000.0,4,주거,2019,1
3,강원도,강릉시,남성,30대,B4,1041289.0,16,개인운송,2019,1
4,강원도,강릉시,여성,40대,B4,426766.0,13,개인운송,2019,1
...,...,...,...,...,...,...,...,...,...,...
90971051,충청북도,충주시,여성,50대,B5,303000.0,17,음식점,2020,4
90971052,충청북도,충주시,여성,60대 이상,B1,100000.0,5,음식점,2020,4
90971053,충청북도,충주시,여성,60대 이상,B2,89000.0,7,음식점,2020,4
90971054,충청북도,충주시,여성,60대 이상,B3,636000.0,29,음식점,2020,4


# 코드화 (메모리 최적화)
- 코드화
  - 가맹점소재지1: 1~17
  - 가맹점소재지2: 1~207
  - 성별: 1,2
  - 연령대별: 1~5
  - 연평균소득추정: 1~11
  - 업종: 1~18

In [4]:
str_list = df['가맹점소재지1'].unique() 

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['가맹점소재지1'] = LE.fit_transform(df['가맹점소재지1'])

int_list = df['가맹점소재지1'].unique()

region1_dict = {key:value for key,value in zip(str_list, int_list)}
region1_dict

{'강원도': 0,
 '경기도': 1,
 '경상남도': 2,
 '경상북도': 3,
 '광주광역시': 4,
 '대구광역시': 5,
 '대전광역시': 6,
 '부산광역시': 7,
 '서울특별시': 8,
 '세종특별자치시': 9,
 '울산광역시': 10,
 '인천광역시': 11,
 '전라남도': 12,
 '전라북도': 13,
 '제주특별자치도': 14,
 '충청남도': 15,
 '충청북도': 16}

In [5]:
str_list = df['가맹점소재지2'].unique() 

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['가맹점소재지2'] = LE.fit_transform(df['가맹점소재지2'])

int_list = df['가맹점소재지2'].unique()

region2_dict = {key:value for key,value in zip(str_list, int_list)}
region2_dict

{'가평군': 0,
 '강남구': 1,
 '강동구': 2,
 '강릉시': 3,
 '강북구': 4,
 '강서구': 5,
 '강진군': 6,
 '강화군': 7,
 '거제시': 8,
 '거창군': 9,
 '경산시': 10,
 '경주시': 11,
 '계룡시': 12,
 '계양구': 13,
 '고령군': 14,
 '고성군': 15,
 '고양시': 16,
 '고창군': 17,
 '고흥군': 18,
 '곡성군': 19,
 '공주시': 20,
 '과천시': 21,
 '관악구': 22,
 '광명시': 23,
 '광산구': 24,
 '광양시': 25,
 '광주시': 26,
 '광진구': 27,
 '괴산군': 28,
 '구례군': 29,
 '구로구': 30,
 '구리시': 31,
 '구미시': 32,
 '군산시': 33,
 '군위군': 34,
 '군포시': 35,
 '금산군': 36,
 '금정구': 37,
 '금천구': 38,
 '기장군': 39,
 '김제시': 40,
 '김천시': 41,
 '김포시': 42,
 '김해시': 43,
 '나주시': 44,
 '남구': 45,
 '남동구': 46,
 '남양주시': 47,
 '남원시': 48,
 '남해군': 49,
 '노원구': 50,
 '논산시': 51,
 '단양군': 52,
 '달서구': 53,
 '달성군': 54,
 '담양군': 55,
 '당진시': 56,
 '대덕구': 57,
 '도봉구': 58,
 '동구': 59,
 '동대문구': 60,
 '동두천시': 61,
 '동래구': 62,
 '동작구': 63,
 '동해시': 64,
 '마포구': 65,
 '목포시': 66,
 '무안군': 67,
 '무주군': 68,
 '문경시': 69,
 '미추홀구': 70,
 '밀양시': 71,
 '보령시': 72,
 '보성군': 73,
 '보은군': 74,
 '봉화군': 75,
 '부산진구': 76,
 '부안군': 77,
 '부여군': 78,
 '부천시': 79,
 '부평구': 80,
 '북구': 81,
 '사상구': 82,
 '사천시': 83,


In [6]:
str_list = df['성별'].unique() 

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['성별'] = LE.fit_transform(df['성별'])

int_list = df['성별'].unique()

gender_dict = {key:value for key,value in zip(str_list, int_list)}
gender_dict

{'남성': 0, '여성': 1}

In [7]:
str_list = df['연령대별'].unique() 

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['연령대별'] = LE.fit_transform(df['연령대별'])

int_list = df['연령대별'].unique()

age_dict = {key:value for key,value in zip(str_list, int_list)}
age_dict

{'20대': 0, '20세 미만': 1, '30대': 2, '40대': 3, '50대': 4, '60대 이상': 5}

In [8]:
str_list = df['연평균소득추정'].unique() 

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['연평균소득추정'] = LE.fit_transform(df['연평균소득추정'])

int_list = df['연평균소득추정'].unique()

income_dict = {key:value for key,value in zip(str_list, int_list)}
income_dict

{'B1': 0,
 'B10': 1,
 'B11': 2,
 'B2': 3,
 'B3': 4,
 'B4': 5,
 'B5': 6,
 'B6': 7,
 'B7': 8,
 'B8': 9,
 'B9': 10}

In [9]:
str_list = df['업종'].unique() 

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['업종'] = LE.fit_transform(df['업종'])

int_list = df['업종'].unique()

store_dict = {key:value for key,value in zip(str_list, int_list)}
store_dict

{'가정용품': 0,
 '개인미용': 1,
 '개인운송': 2,
 '교육서비스': 3,
 '기타': 4,
 '대형판매': 5,
 '보건': 6,
 '보험': 7,
 '숙박서비스': 8,
 '식료품': 9,
 '여객운송': 10,
 '오락_문화': 11,
 '음식점': 12,
 '의류_잡화': 13,
 '전자상거래': 14,
 '정보통신': 15,
 '종합소매': 16,
 '주거': 17}

In [10]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)                    
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                    elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                        df[col] = df[col].astype(np.uint64)
                elif str(col_type)[:5] == 'float':
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

        elif col_type == object:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

df = reduce_mem_usage(df)

df

Memory usage of dataframe is 6003.03 MB
Memory usage after optimization is: 2198.29 MB
Decreased by 63.4%


,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,이용금액,이용건수,업종,년,월
0,0,3,1,3,3,1070000.0,4,3,2019,1
1,0,3,0,0,3,48000.0,4,1,2019,1
2,0,3,0,4,5,735000.0,4,17,2019,1
3,0,3,0,2,5,1041289.0,16,2,2019,1
4,0,3,1,3,5,426766.0,13,2,2019,1
...,...,...,...,...,...,...,...,...,...,...
90971051,16,183,1,4,6,303000.0,17,12,2020,4
90971052,16,183,1,5,0,100000.0,5,12,2020,4
90971053,16,183,1,5,3,89000.0,7,12,2020,4
90971054,16,183,1,5,4,636000.0,29,12,2020,4


In [11]:
# 코드화 dict 저장

import pickle

with open(data_dir + 'dict_pickle/region1_dict.pickle','wb') as fw:
    pickle.dump(region1_dict, fw)
with open(data_dir + 'dict_pickle/region2_dict.pickle','wb') as fw:
    pickle.dump(region2_dict, fw)
with open(data_dir + 'dict_pickle/gender_dict.pickle','wb') as fw:
    pickle.dump(gender_dict, fw)
with open(data_dir + 'dict_pickle/age_dict.pickle','wb') as fw:
    pickle.dump(age_dict, fw)
with open(data_dir + 'dict_pickle/income_dict.pickle','wb') as fw:
    pickle.dump(income_dict, fw)
with open(data_dir + 'dict_pickle/store_dict.pickle','wb') as fw:
    pickle.dump(store_dict, fw)


# 데이터 저장

In [12]:
""" col 순서 재배치 """
df = df[['년', '월', '가맹점소재지1', '가맹점소재지2', '성별', '연령대별', '연평균소득추정', 
         '이용금액', '이용건수', '업종']]
         
df.reset_index().to_feather(data_dir + 'data_1901_2004_classified_2.ftr')

In [ ]:
df.reset_index()

,index,년,월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,이용금액,이용건수,업종
0,0,2019,1,0,3,1,3,3,1070000.0,4,19
1,1,2019,1,0,3,0,0,3,48000.0,4,5
2,2,2019,1,0,3,0,4,5,735000.0,4,1
3,3,2019,1,0,3,0,2,5,1041289.0,16,15
4,4,2019,1,0,3,1,3,5,426766.0,13,15
...,...,...,...,...,...,...,...,...,...,...,...
88656830,90971051,2020,4,16,183,1,4,6,303000.0,17,9
88656831,90971052,2020,4,16,183,1,5,0,100000.0,5,9
88656832,90971053,2020,4,16,183,1,5,3,89000.0,7,9
88656833,90971054,2020,4,16,183,1,5,4,636000.0,29,9
